In [16]:
import requests
import json
from tqdm import tqdm
import pandas as pd
import numpy as np
import re

In [17]:
DATA_PATH = './data/'
DATA = DATA_PATH + 'BindingDB_All.tsv'

## Let's look for a suitable protein family to study

Step 1. Match targets to their families. We made a choice to use PANTHER classification, although other classifications exist too.

In [ ]:
df = pd.read_csv(DATA, sep="\t", on_bad_lines='skip', low_memory=False, dtype={
    "BindingDB Reactant_set_id": np.int32,
})


In [ ]:
df

,BindingDB Reactant_set_id,Ligand SMILES,Ligand InChI,Ligand InChI Key,BindingDB MonomerID,BindingDB Ligand Name,Target Name,Target Source Organism According to Curator or DataSource,Ki (nM),IC50 (nM),...,UniProt (SwissProt) Recommended Name of Target Chain.12,UniProt (SwissProt) Entry Name of Target Chain.12,UniProt (SwissProt) Primary ID of Target Chain.12,UniProt (SwissProt) Secondary ID(s) of Target Chain.12,UniProt (SwissProt) Alternative ID(s) of Target Chain.12,UniProt (TrEMBL) Submitted Name of Target Chain.12,UniProt (TrEMBL) Entry Name of Target Chain.12,UniProt (TrEMBL) Primary ID of Target Chain.12,UniProt (TrEMBL) Secondary ID(s) of Target Chain.12,UniProt (TrEMBL) Alternative ID(s) of Target Chain.12
0,1,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(CCCCCC(O)=...,InChI=1S/C31H42N2O7/c34-27(35)17-9-3-11-19-32-...,XGEGDSLAQZJGCW-HHGOQMMWSA-N,608734,"6-[(4R,5S,6S,7R)-4,7-dibenzyl-3-(5-carboxypent...",Dimer of Gag-Pol polyprotein [501-599],Human immunodeficiency virus 1,0.24,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(C\C=C\c2cn...,InChI=1S/C31H34N6O3/c38-29-27(17-23-9-3-1-4-10...,UZLMEAPBHYEHAC-UNTBESQGSA-N,22,"(4R,5S,6S,7R)-4,7-dibenzyl-5,6-dihydroxy-1,3-b...",Dimer of Gag-Pol polyprotein [501-599],Human immunodeficiency virus 1,0.25,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(CC2CC2)C(=...,InChI=1S/C29H34N4O3/c34-27-25(16-21-8-3-1-4-9-...,HYNYUFZPPJMPOB-UTWJFGBXSA-N,23,"(4R,5S,6S,7R)-4,7-dibenzyl-1-(cyclopropylmethy...",Dimer of Gag-Pol polyprotein [501-599],Human immunodeficiency virus 1,0.41,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,OCCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@...,InChI=1S/C29H40N2O4/c32-18-10-2-1-9-17-30-25(1...,YXVAZXDWVZTGGD-VIJSPRBVSA-N,24,"(4R,5S,6S,7R)-4,7-dibenzyl-1-(cyclopropylmethy...",Dimer of Gag-Pol polyprotein [501-599],Human immunodeficiency virus 1,0.8,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,OCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@H...,InChI=1S/C28H38N2O4/c31-17-9-3-8-16-29-24(18-2...,WWTSWTPNILRSJX-XDZXDJIYSA-N,25,"(4R,5S,6S,7R)-4,7-dibenzyl-1-(cyclopropylmethy...",Dimer of Gag-Pol polyprotein [501-599],Human immunodeficiency virus 1,0.99,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2927359,51531022,CC[C@H]1CN(Cc2cc(C)cc(CC(O)=O)c2)CCN1c1nc2ccc(...,InChI=1S/C24H26F3N3O2S/c1-3-19-14-29(13-17-9-1...,ZBKXSRSXIGZPCY-IBGZPJMESA-N,50615961,CHEMBL5268424,Peroxisome proliferator-activated receptor gamma,Mus musculus,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2927360,51531023,CC[C@@H]1CN(Cc2cc(C)cc(CC(O)=O)c2)CCN1c1nc2ccc...,InChI=1S/C24H26F3N3O2S/c1-3-19-14-29(13-17-9-1...,ZBKXSRSXIGZPCY-LJQANCHMSA-N,50615962,CHEMBL5272996,Peroxisome proliferator-activated receptor gamma,Mus musculus,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2927361,51531024,CC(C)[C@@H]1CN(Cc2cc(C)cc(CC(O)=O)c2)CCN1c1nc2...,InChI=1S/C25H28F3N3O2S/c1-15(2)21-14-30(13-18-...,RVZUBLMVVIWYLZ-NRFANRHFSA-N,50615964,CHEMBL5277165,Peroxisome proliferator-activated receptor gamma,Mus musculus,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2927362,51531025,COc1ccc(cc1)N(C)c1nc(C)nc2[nH]ccc12,InChI=1S/C15H16N4O/c1-10-17-14-13(8-9-16-14)15...,HKPVMULFKBBABY-UHFFFAOYSA-N,50615967,CHEMBL1275943,Similar to alpha-tubulin isoform 1,NaN,NaN,2600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import sys
sys.path.append('./src/utils')
from retrieve_family import retrieve_family

uniprot_ids = df["UniProt (SwissProt) Entry Name of Target Chain"].unique()
# Retrieve family information for the test ID
family_dict = await retrieve_family(uniprot_ids)


100%|██████████| 6533/6533 [00:04<00:00, 1324.54it/s]


In [ ]:
#map each list to a string

# List of all unique protein families
families = set()
for key in family_dict:
    families.update(family_dict[key])

# Create a dictionary with the protein families as keys and the proteins as values
family_dict = {family: [] for family in families}
for key in family_dict:
    for protein in family_dict:
        if protein in family_dict[key]:
            family_dict[key].append(protein)

for key in family_dict:
    family_dict[key] = '||'.join(family_dict[key])

with open("protein_families.json", "w") as f:
    json.dump(family_dict, f, indent=2)
    print("Protein families saved to 'protein_families.json'")


Protein families saved to 'protein_families.json'


In [ ]:
df_family = pd.DataFrame.from_dict(family_dict, orient='index', columns=['Family'])
df_family


,Family
TRANSCRIPTION FACTOR SPT20 HOMOLOG,
DNA POLYMERASE III SUBUNIT BETA,
GLUTAMATE RECEPTOR-INTERACTING PROTEIN GRIP,
NUCLEOPORIN NUP188 HOMOLOG,
SOLUTE CARRIER FAMILY 35,
...,...
TESTICULAR SOLUBLE ADENYLYL CYCLASE,
ALSIN HOMOLOG,
ENHANCER OF MRNA-DECAPPING PROTEIN 4,
TRANSMEMBRANE PROTEIN 139,


In [ ]:
df_family = pd.DataFrame.from_dict(family_dict, orient='index', columns=['Family'])
# Number of unique protein families
print(f"There are {len(families)} unique families in this dataset")
print(f"Number of NaNs in 'Family' column: {df_family['Family'].isna().sum()}")

# Map for each family the number of proteins in the dataset (Family column is a || separated string)
family_counts = {}
for family in families:
    if family not in family_counts:
        family_counts[family] = 0
    family_counts[family] += int(df_family['Family'].str.contains(re.escape(family)).sum())



There are 5307 unique families in this dataset
Number of NaNs in 'Family' column: 0


In [ ]:
print(f"Number of proteins in each family in descending order:")
family_counts = dict(sorted(family_counts.items(), key=lambda item: item[1], reverse=True))
for family in family_counts:
    print(f"{family}: {family_counts[family]}")

Number of proteins in each family in descending order:
TRANSCRIPTION FACTOR SPT20 HOMOLOG: 0
DNA POLYMERASE III SUBUNIT BETA: 0
GLUTAMATE RECEPTOR-INTERACTING PROTEIN GRIP: 0
NUCLEOPORIN NUP188 HOMOLOG: 0
SOLUTE CARRIER FAMILY 35: 0
ADIPOCYTE-SPECIFIC PROTEIN 4-RELATED: 0
ZONA PELLUCIDA SPERM-BINDING PROTEIN 3: 0
SPERMATOGENESIS-ASSOCIATED PROTEIN 24: 0
GEO13361P1-RELATED: 0
SERINE/THREONINE-PROTEIN KINASE: 0
RGPR-RELATED: 0
UDP-GLUCOSE 4-EPIMERASE: 0
REGULATOR OF CELL CYCLE RGCC: 0
STANNIOCALCIN: 0
FALZ-RELATED BROMODOMAIN-CONTAINING PROTEINS: 0
KUNITZ-TYPE PROTEASE INHIBITOR-RELATED: 0
DNA HELICASE RECQ FAMILY MEMBER: 0
DAN DOMAIN FAMILY MEMBER 5: 0
DNAJ HOMOLOG SUBFAMILY C MEMBER 8/LIPOPOLYSACCHARIDE SPECIFIC RESPONSE-7-RELATED: 0
HYDRANTOINASE/DIHYDROPYRIMIDINASE FAMILY MEMBER: 0
VON WILLEBRAND FACTOR A DOMAIN-CONTAINING PROTEIN 5A: 0
COPPER AMINE OXIDASE: 0
CEROID-LIPOFUSCINOSIS NEURONAL PROTEIN 6: 0
RING ZINC FINGER PROTEIN: 0
IMMUNOGLOBULIN FC RECEPTOR: 0
CULLIN AND GALACTOSE-BI

In [ ]:
# Only keep families with at least n targets
n = 5
df_family_filtered = df_family[df_family['Family'].apply(lambda x: any([family_counts[family] >= n for family in x.split('||')]))]
print(f"Number of families with at least {n} targets: {len(df_family_filtered)}")

KeyError: ''

In [ ]:
# In the original dataframe, only keep the rows with UniProt IDs that have a family
df_filtered = df[df['UniProt (SwissProt) Entry Name of Target Chain'].isin(df_family.index)]
# Add the family column to the original dataframe
df_filtered = df_filtered.join(df_family, on='UniProt (SwissProt) Entry Name of Target Chain')

In [ ]:
# Check the amount of missing Ki
print(f"Number of NaNs in 'Ki (nM)' column: {df_filtered['Ki (nM)'].isna().sum()}")
# Check the amount of missing IC50
print(f"Number of NaNs in 'IC50 (nM)' column: {df_filtered['IC50 (nM)'].isna().sum()}")
# Drop the rows with missing IC50
df_filtered = df_filtered.dropna(subset=['IC50 (nM)'])

Number of NaNs in 'Ki (nM)' column: 2313133
Number of NaNs in 'IC50 (nM)' column: 960211


In [ ]:
# family_dict = family -> [protein1, protein2, protein3]
# df_grouped = family -> (each column of df_filtered counts the number of proteins in the family)
# df filtered only has a || separated list of families

dict_grouped = {}
for row in tqdm(df_filtered.iterrows(), total=len(df_filtered)):
    row = row[1]
    families = row['Family'].split('||')
    for family in families:
        if family not in dict_grouped:
            dict_grouped[family] = {'IC50 (nM)': 0, 'UniProt IDs': 0}
        dict_grouped[family]['IC50 (nM)'] += 1
        dict_grouped[family]['UniProt IDs'] += 1



  0%|          | 0/1939305 [00:00<?, ?it/s]

: 

In [ ]:
df_grouped = pd.DataFrame.from_dict(dict_grouped, orient='index')
df_grouped['Ligand to Target Ratio'] = df_grouped['IC50 (nM)'] / df_grouped['UniProt IDs']
df_grouped = df_grouped.sort_values('Ligand to Target Ratio', ascending=False)
df_grouped

,IC50 (nM),UniProt IDs,Ligand to Target Ratio
H2.0-LIKE HOMEOBOX PROTEIN,4,4,1.0
CYTOCHROME P450 FAMILY 3,12444,12444,1.0
CYTOCHROME P450 508A4-RELATED,43461,43461,1.0
MEVALONATE KINASE/GALACTOKINASE,1484,1484,1.0
ALDOSE-1-EPIMERASE,409,409,1.0
...,...,...,...
DNA REPLICATION FACTOR CDT1,974,974,1.0
TGF-BETA FAMILY,44822,44822,1.0
DUAL SPECIFICITY MITOGEN-ACTIVATED PROTEIN KINASE KINASE DSOR1-LIKE PROTEIN,37952,37952,1.0
DUAL SPECIFICITY MITOGEN-ACTIVATED PROTEIN KINASE KINASE 5-RELATED,2549,2549,1.0


In [ ]:
# Save the filtered dataframe to a new file
df_grouped.to_csv(DATA_PATH + 'BindingDB_counts.tsv', sep='\t', index=False)

In [ ]:
# Show proteins belonging to PHOSPHOINOSITIDE-SPECIFIC PHOSPHOLIPASE C FAMILY PROTEIN
df_filtered[df_filtered['Family'] == 'PHOSPHOINOSITIDE-SPECIFIC PHOSPHOLIPASE C FAMILY PROTEIN']['UniProt (SwissProt) Entry Name of Target Chain'].unique()

array(['BTK_HUMAN', 'PLCG1_BOVIN', 'PLCB1_HUMAN', 'PLCB_HUMAN'],
      dtype=object)

In [ ]:
# Find "PLC" in UniProt IDs
winners = df_filtered[df_filtered['UniProt (SwissProt) Entry Name of Target Chain'].str.contains('PLC', na=False)]['UniProt (SwissProt) Entry Name of Target Chain'].unique()

In [ ]:
# Compute the number of occurencies of each winner
test = ['BTK_HUMAN']
df_filtered[df_filtered['UniProt (SwissProt) Entry Name of Target Chain'].isin(test)].groupby('UniProt (SwissProt) Entry Name of Target Chain').count()

,BindingDB Reactant_set_id,Ligand SMILES,Ligand InChI,Ligand InChI Key,BindingDB MonomerID,BindingDB Ligand Name,Target Name,Target Source Organism According to Curator or DataSource,Ki (nM),IC50 (nM),...,UniProt (SwissProt) Entry Name of Target Chain.12,UniProt (SwissProt) Primary ID of Target Chain.12,UniProt (SwissProt) Secondary ID(s) of Target Chain.12,UniProt (SwissProt) Alternative ID(s) of Target Chain.12,UniProt (TrEMBL) Submitted Name of Target Chain.12,UniProt (TrEMBL) Entry Name of Target Chain.12,UniProt (TrEMBL) Primary ID of Target Chain.12,UniProt (TrEMBL) Secondary ID(s) of Target Chain.12,UniProt (TrEMBL) Alternative ID(s) of Target Chain.12,Family
UniProt (SwissProt) Entry Name of Target Chain,,,,,,,,,,,,,,,,,,,,,
BTK_HUMAN,22095,22095,16849,16849,22095,22095,22095,19969,0,22095,...,0,0,0,0,0,0,0,0,0,22095


In [ ]:
response = requests.get(f"https://rest.uniprot.org/uniprotkb/search?query=BTK_HUMAN&fields=xref_panther",
                        headers={"Accept": "application/json"})
if response.status_code == 200:
    results = response.json().get("results", [])
results

[{'entryType': 'UniProtKB reviewed (Swiss-Prot)',
  'primaryAccession': 'Q06187',
  'uniProtKBCrossReferences': [{'database': 'PANTHER',
    'id': 'PTHR24418',
    'properties': [{'key': 'EntryName', 'value': 'TYROSINE-PROTEIN KINASE'},
     {'key': 'MatchStatus', 'value': '1'}]},
   {'database': 'PANTHER',
    'id': 'PTHR24418:SF92',
    'properties': [{'key': 'EntryName',
      'value': 'TYROSINE-PROTEIN KINASE BTK'},
     {'key': 'MatchStatus', 'value': '1'}]}],
  'extraAttributes': {'uniParcId': 'UPI0000126B16'}},
 {'entryType': 'UniProtKB reviewed (Swiss-Prot)',
  'primaryAccession': 'P78347',
  'uniProtKBCrossReferences': [{'database': 'PANTHER',
    'id': 'PTHR46304:SF2',
    'properties': [{'key': 'EntryName',
      'value': 'GENERAL TRANSCRIPTION FACTOR II-I'},
     {'key': 'MatchStatus', 'value': '1'}]},
   {'database': 'PANTHER',
    'id': 'PTHR46304',
    'properties': [{'key': 'EntryName',
      'value': 'GENERAL TRANSCRIPTION FACTOR II-I REPEAT DOMAIN-CONTAINING PROTEIN 1